In [7]:
import pandas as pd

df = pd.read_csv("diabetes.csv")
df.head()

,6,148,72,35,0,33.6,0.627,50,Outcome
0,1,85,66,29,0,26.6,0.351,31,0
1,8,183,64,0,0,23.3,0.672,32,1
2,1,89,66,23,94,28.1,0.167,21,0
3,0,137,40,35,168,43.1,2.288,33,1
4,5,116,74,0,0,25.6,0.201,30,0


In [8]:
df.isnull().sum()

6          0
148        0
72         0
35         0
0          0
33.6       0
0.627      0
50         0
Outcome    0
dtype: int64

In [9]:
df.describe()

,6,148,72,35,0,33.6,0.627,50,Outcome
count,767.000000,767.000000,767.000000,767.000000,767.000000,767.000000,767.000000,767.000000,767.000000
mean,3.842243,120.859192,69.101695,20.517601,79.903520,31.990482,0.471674,33.219035,0.348110
std,3.370877,31.978468,19.368155,15.954059,115.283105,7.889091,0.331497,11.752296,0.476682
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243500,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,32.000000,32.000000,0.371000,29.000000,0.000000
75%,6.000000,140.000000,80.000000,32.000000,127.500000,36.600000,0.625000,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [10]:
df.Outcome.value_counts()

0    500
1    267
Name: Outcome, dtype: int64

In [11]:
X = df.drop("Outcome",axis="columns")
y = df.Outcome

In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled[:3]

array([[-0.84372629, -1.12208597, -0.16024856,  0.53202348, -0.69355921,
        -0.68372895, -0.36426474, -0.18894038],
       [ 1.23423997,  1.94447577, -0.26357823, -1.28688187, -0.69355921,
        -1.10230105,  0.60470064, -0.1037951 ],
       [-0.84372629, -0.99692019, -0.16024856,  0.15569823,  0.12235685,
        -0.49346891, -0.91968415, -1.0403932 ]])

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, stratify=y, random_state=10)

In [14]:
X_train.shape

(575, 8)

In [15]:
X_test.shape

(192, 8)

In [16]:
y_train.value_counts()

0    375
1    200
Name: Outcome, dtype: int64

In [17]:
y_test.value_counts()

0    125
1     67
Name: Outcome, dtype: int64

In [18]:
67/125

0.536

## Train using stand alone model 

In [19]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

scores = cross_val_score(DecisionTreeClassifier(), X, y, cv=5)
scores

array([0.7012987 , 0.66233766, 0.71895425, 0.79738562, 0.75163399])

In [20]:
scores.mean()

0.7263220439691029

## Train using Bagging

In [21]:
from sklearn.ensemble import BaggingClassifier

bag_model = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(), 
    n_estimators=100, 
    max_samples=0.8, 
    oob_score=True,
    random_state=0
)
bag_model.fit(X_train, y_train)
bag_model.oob_score_

0.7634782608695653

In [22]:
bag_model.score(X_test, y_test)

0.7552083333333334

In [23]:
bag_model = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(), 
    n_estimators=100, 
    max_samples=0.8, 
    oob_score=True,
    random_state=0
)
scores = cross_val_score(bag_model, X, y, cv=5)
scores

array([0.75324675, 0.73376623, 0.75816993, 0.81699346, 0.73202614])

In [24]:
scores.mean()

0.7588405058993294

We can see some improvement in test score with bagging classifier as compared to a standalone classifier

## Train using Random Forest

In [25]:
from sklearn.ensemble import RandomForestClassifier

scores = cross_val_score(RandomForestClassifier(n_estimators=50), X, y, cv=5)
scores.mean()

0.7719378660555132